# Load Data

In [27]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
df = pd.read_csv("train.csv")

X = df.iloc[:, 0:len(df.columns)-1]
X = pd.get_dummies(X)

Y = df.loc[:, 'target']

### Boxcox transform the Target Column

In [16]:
Y_boxcox, lmda = boxcox(Y)

Use inv_boxcox to transform predicitions back into the native scale

In [33]:
inv_boxcox(Y_boxcox, lmda)

array([21.639999, 19.919998, 21.150001, ..., 23.36    , 20.1407  ,
       21.3     ])

### Make Train/Test Split

In [17]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size = 0.20, random_state = 42)
train_X_bc, val_X_bc, train_Y_bc, val_Y_bc = train_test_split(X, Y, test_size = 0.20, random_state = 42)

### Get baseline predictions for each transformation of Target

Essentially, guess the average of the column

In [35]:
baseline_preds = val_Y.mean()
baseline_errors = abs(val_Y - baseline_preds)
print('Average baseline MAE: ', round(np.mean(baseline_errors), 2))

baseline_preds_bc = val_Y_bc.mean()
baseline_errors_bc = abs(val_Y_bc - baseline_preds_bc)
print('BoxCox Average baseline MAE: ', round(np.mean(baseline_errors_bc), 2))


Average baseline MAE:  0.93
BoxCox Average baseline MAE:  0.93


# Lasso

In [97]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

In [66]:
tuned_parameters = [{'alpha': [0.4, 0.5, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.7, 0.8, 0.9, 1]}]

Find the best gamma for both normal and boxcox transformed 

In [68]:
clf = GridSearchCV(Lasso(), tuned_parameters, cv=5,
                       scoring='neg_mean_absolute_error')
clf.fit(train_X, train_Y)
clf.best_params_

{'alpha': 0.62}

In [67]:
clf_bc =  GridSearchCV(Lasso(), tuned_parameters, cv=5,
                       scoring='neg_mean_absolute_error')
clf.fit(train_X_bc, train_Y_bc)
clf.best_params_

{'alpha': 0.62}

In [88]:
lasso_fit = Lasso(0.62, random_state=450, tol=0.0000000001)
lasso_fit = lasso_fit.fit(train_X, train_Y)

predictions = lasso_fit.predict(val_X)

# Check GOF, Calculate MAE
errors = abs(val_Y - predictions)

print('Mean Absolute Error:', round(np.mean(errors), 2))

coef = lasso_fit.coef_

lasso_features = train_X.columns[coef != 0]

Mean Absolute Error: 0.93


In [89]:
lasso_features

Index(['num1', 'num30', 'num37', 'num59'], dtype='object')

In [79]:
lasso_fit_bc = lasso_fit.fit(train_X_bc, train_Y_bc)

predictions = lasso_fit_bc.predict(val_X_bc)

# Check GOF, Calculate MAE
errors = abs(val_Y_bc - predictions)
errors_trans = abs(val_Y - inv_boxcox(predictions, lmda))

print('MAE BoxCox:', round(np.mean(errors), 2))
print('MAE BoxCox, Transformed:', round(np.mean(errors_trans), 2))


# Check out non-zero coefficients 
coef = lasso_fit_bc.coef_

#train_X.columns[coef != 0]

MAE BoxCox: 0.93
MAE BoxCox, Transformed: 11.78


# XgBoost

In [93]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score

Use a hyperparameter grid search: https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

In [114]:
params = {
        'min_child_weight': [1, 2, 4],
        'gamma': [0, 0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1],
        'colsample_bytree': [0.5, 0.7, 0.9],
        'max_depth': [3, 5, 6, 8]
        }

In [111]:
xgb = XGBRegressor(learning_rate=0.2, n_estimators=600, objective='reg:linear', eval_metric='mae',
                    silent=True)

In [117]:
folds = 5
param_comb = 50

kf = KFold(n_splits=folds,shuffle=True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb,
                                   scoring='neg_mean_absolute_error', n_jobs=4,
                                   cv=kf.split(train_X,train_Y), verbose=3, random_state=1001 )

random_search.fit(train_X, train_Y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 15.5min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 31.2min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000000957E6AB570>,
          error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='mae', gamma=0, learning_rate=0.2,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=600, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=50, n_jobs=4,
          param_distributions={'min_child_weight': [1, 2, 4], 'gamma': [0, 0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1], 'colsample_bytree': [0.5, 0.7, 0.9], 'max_depth': [3, 5, 6, 8]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=3)

In [118]:
random_search.best_params_

{'colsample_bytree': 0.7,
 'gamma': 5,
 'max_depth': 3,
 'min_child_weight': 2,
 'subsample': 1}

In [119]:
predictions = random_search.predict(val_X)

# Check GOF, Calculate MAE
errors = abs(val_Y - predictions)
errors_trans = abs(val_Y - inv_boxcox(predictions, lmda))

print('MAE BoxCox:', round(np.mean(errors), 2))
print('MAE BoxCox, Transformed:', round(np.mean(errors_trans), 2))


# Check out non-zero coefficients 
#coef = lasso_fit_bc.coef_

#train_X.columns[coef != 0]

MAE BoxCox: 0.98
MAE BoxCox, Transformed: 11.78


Use the best params to fit some models

In [123]:
xgb = XGBRegressor(learning_rate=0.1, n_estimators=1000, objective='reg:linear', eval_metric='mae',
                    silent=True, **random_search.best_params_ )

In [126]:
xgb.fit(train_X, train_Y)
predictions = xgb.predict(val_X)
errors = abs(val_Y - predictions)
print("MAE: ", round(np.mean(errors), 2))

MAE:  0.97


BoxCox

In [127]:
xgb.fit(train_X_bc, train_Y_bc)
predictions = xgb.predict(val_X_bc)
errors = abs(val_Y_bc - predictions)
errors_trans = abs(val_Y - inv_boxcox(predictions, lmda))
print("MAE Boxcox: ", round(np.mean(errors), 2))
print("MAE un-Boxcox: ", round(np.mean(errors_trans), 2))

MAE Boxcox:  0.97
MAE un-Boxcox:  11.78


# Simple Neural Network

### First, We must standardize the input to be between 1 and 0

Also, let's start off with only the features identified to be useful to lasso!

In [152]:
def range_std(column):
    # Helper function, range standardizes a list of values
    return (column - column.min())/(column.max() - column.min())

def range_std_reversible(column):
    # Helper function, range standardizes a list of values
    # Returns min and max for later reversal
    return (column - column.min())/(column.max() - column.min()), column.min(), column.max()

def range_std_reverse(column, min_val, max_value):
    return (column*(max_value - min_val) + min_val)

In [137]:
features_to_standardize = [*lasso_features, 'target']
df_thin = df.loc[:, features_to_standardize]
for column_name in features_to_standardize:
    df_thin[column_name] = range_std(df_thin[column_name])
    
df_thin.describe()

Split features and target column

In [142]:
# Use these as our training set
trainY_std = df_thin.loc[:, 'target']
trainX_std = df_thin.drop(columns='target')

In [201]:
# Boxcox transform
trainY_std_bc, min_value, max_value = range_std_reversible(Y_boxcox)

In [183]:
train_X_std, val_X_std, train_Y_std, val_Y_std = train_test_split(trainX_std, trainY_std, test_size = 0.20, random_state = 42)
train_X_std_bc, val_X_std_bc, train_Y_std_bc, val_Y_std_bc = train_test_split(trainX_std, trainY_std_bc, test_size = 0.20, random_state = 42)

In [162]:
# This is how we will return the range standardized, boxcox transformed values to their original scale
inv_boxcox(range_std_reverse(trainY_std_bc, min_value, max_value), lmda)

array([21.639999, 19.919998, 21.150001, ..., 23.36    , 20.1407  ,
       21.3     ])

### Baseline Predictions

In [230]:
std_error = abs(range_std_reverse(val_Y_std, df.target.min(), df.target.max()) - range_std_reverse(val_Y_std.mean(), df.target.min(), df.target.max()))
print("MAE:", np.mean(std_error))

std_error_bc = abs(inv_boxcox(range_std_reverse(val_Y_std_bc, Y_boxcox.min(), Y_boxcox.max()), lmda) - inv_boxcox(range_std_reverse(val_Y_std_bc.mean(), Y_boxcox.min(), Y_boxcox.max()), lmda))
print("MAE, bc:", np.mean(std_error_bc))


MAE: 0.9336471051621303
MAE, bc: 0.9324778352168116


## Simple Nueral

In [178]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.pipeline import Pipeline
from keras.constraints import maxnorm

In [274]:
def simplemodel():
    model = Sequential()
    
    model.add(Dense(100, input_shape= (4,), kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(4)))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(4)))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, kernel_initializer='normal'))

    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    return model

In [260]:
snn = KerasRegressor(build_fn=simplemodel, epochs=100, verbose=True)
snn.fit(train_X_std, train_Y_std)

Epoch 1/100
5080/5080 [==============================] - 2s 332us/step - loss: 0.1367
Epoch 2/100
5080/5080 [==============================] - 0s 65us/step - loss: 0.0642
Epoch 3/100
5080/5080 [==============================] - 0s 62us/step - loss: 0.0603
Epoch 4/100
5080/5080 [==============================] - 0s 64us/step - loss: 0.0555
Epoch 5/100
5080/5080 [==============================] - 0s 64us/step - loss: 0.0517
Epoch 6/100
5080/5080 [==============================] - 0s 64us/step - loss: 0.0495
Epoch 7/100
5080/5080 [==============================] - 0s 66us/step - loss: 0.0474
Epoch 8/100
5080/5080 [==============================] - 0s 65us/step - loss: 0.0457
Epoch 9/100
5080/5080 [==============================] - 0s 65us/step - loss: 0.0444
Epoch 10/100
5080/5080 [==============================] - 0s 66us/step - loss: 0.0431
Epoch 11/100
5080/5080 [==============================] - 0s 64us/step - loss: 0.0418
Epoch 12/100
5080/5080 [==============================] - 0s 6

5080/5080 [==============================] - 0s 63us/step - loss: 0.0395
Epoch 97/100
5080/5080 [==============================] - 0s 60us/step - loss: 0.0395
Epoch 98/100
5080/5080 [==============================] - 0s 61us/step - loss: 0.0395
Epoch 99/100
5080/5080 [==============================] - 0s 62us/step - loss: 0.0396
Epoch 100/100
5080/5080 [==============================] - 0s 60us/step - loss: 0.0395


In [261]:
preds = snn.predict(val_X_std)
preds = range_std_reverse(preds, df.target.min(), df.target.max())
actual = range_std_reverse(val_Y_std, df.target.min(), df.target.max())

errors = abs(actual - preds)

print("MAE: ", round(np.mean(errors), 5))

1270/1270 [==============================] - 1s 453us/step
MAE:  0.93963


### Boxcox transformed

In [248]:
snn_std = KerasRegressor(build_fn=simplemodel, epochs=100, verbose=True)
snn.fit(train_X_std_bc, train_Y_std_bc)

Epoch 1/100
5080/5080 [==============================] - 1s 293us/step - loss: 0.1544
Epoch 2/100
5080/5080 [==============================] - 0s 52us/step - loss: 0.0534
Epoch 3/100
5080/5080 [==============================] - 0s 54us/step - loss: 0.0502
Epoch 4/100
5080/5080 [==============================] - 0s 52us/step - loss: 0.0477
Epoch 5/100
5080/5080 [==============================] - 0s 55us/step - loss: 0.0457
Epoch 6/100
5080/5080 [==============================] - 0s 53us/step - loss: 0.0443
Epoch 7/100
5080/5080 [==============================] - 0s 51us/step - loss: 0.0429
Epoch 8/100
5080/5080 [==============================] - 0s 53us/step - loss: 0.0420
Epoch 9/100
5080/5080 [==============================] - 0s 56us/step - loss: 0.0414
Epoch 10/100
5080/5080 [==============================] - 0s 53us/step - loss: 0.0409
Epoch 11/100
5080/5080 [==============================] - 0s 54us/step - loss: 0.0406
Epoch 12/100
5080/5080 [==============================] - 0s 5

5080/5080 [==============================] - 0s 55us/step - loss: 0.0391
Epoch 97/100
5080/5080 [==============================] - 0s 52us/step - loss: 0.0391
Epoch 98/100
5080/5080 [==============================] - 0s 51us/step - loss: 0.0391
Epoch 99/100
5080/5080 [==============================] - 0s 50us/step - loss: 0.0391
Epoch 100/100
5080/5080 [==============================] - 0s 51us/step - loss: 0.0391


In [250]:
preds_raw = snn.predict(val_X_std_bc)
preds = inv_boxcox(range_std_reverse(preds_raw, Y_boxcox.min(), Y_boxcox.max()), lmda)
actual = inv_boxcox(range_std_reverse(val_Y_std_bc, min_value, max_value), lmda)

errors = abs(actual - preds)

print("MAE : ", round(np.mean(abs(val_Y_std_bc - preds_raw)), 2))
print("MAE Native Scale: ", round(np.mean(errors), 5))

1270/1270 [==============================] - 0s 35us/step
MAE :  0.04
MAE Native Scale:  0.94339


# Use Normal Column

In [296]:
norm_data = pd.read_csv("target.csv")

In [297]:
lasso_features

Index(['num1', 'num30', 'num37', 'num59'], dtype='object')

In [298]:
features_to_standardize = [*lasso_features, 'target']

df_thin = norm_data.loc[:, features_to_standardize]
df_thin.target = lambertw(df_thin.target)

for column_name in features_to_standardize:
    df_thin[column_name] = range_std(df_thin[column_name])
    
df_thin.describe()

,num1,num30,num37,num59,target
count,6350.000000,6350.000000,6350.000000,6350.000000,(6350+0j)
mean,0.097565,0.891458,0.735050,0.500000,(0.6168195605350324+0j)
std,0.114833,0.062245,0.104127,0.288743,(0.0551010880080418+0j)
min,0.000000,0.000000,0.000000,0.000000,0j
25%,0.007875,0.862720,0.688233,0.250000,(0.5963608384885867+0j)
50%,0.064139,0.904846,0.749047,0.500000,(0.61959382335273+0j)
75%,0.143567,0.935322,0.799315,0.750000,(0.6424944271345217+0j)
max,1.000000,1.000000,1.000000,1.000000,(1+0j)


In [299]:
# Use these as our training set
trainY_std = df_thin.loc[:, 'target']
trainX_std = df_thin.loc[:,lasso_features]

In [300]:
train_X_std, val_X_std, train_Y_std, val_Y_std = train_test_split(trainX_std, trainY_std, test_size = 0.20, random_state = 42)


In [301]:
abs(val_Y_std - val_Y_std.mean())

730     0.050629
2131    0.006585
3070    0.004074
4283    0.019692
217     0.107404
3545    0.009654
4080    0.016731
3674    0.011104
6176    0.100648
4615    0.025217
3459    0.008531
4373    0.021169
465     0.072265
5703    0.058328
2981    0.003272
1183    0.030178
4040    0.015989
3680    0.011149
1263    0.027436
3622    0.010403
3214    0.005530
746     0.049625
1611    0.017378
4684    0.026684
80      0.152974
3784    0.012268
2882    0.002307
1616    0.017331
6296    0.143494
3612    0.010402
          ...   
509     0.067894
5772    0.061850
2153    0.006585
4851    0.030706
748     0.049625
1617    0.017331
5271    0.041216
2404    0.003159
5986    0.078550
3313    0.006657
2024    0.008494
4546    0.024115
5556    0.051596
2181    0.005990
471     0.072009
5139    0.037516
3961    0.014874
2619    0.000645
5462    0.048389
5996    0.079410
3096    0.004402
896     0.041879
4796    0.028880
251     0.097987
1209    0.029393
3525    0.009280
3942    0.014595
2895    0.0025

In [302]:
std_error = abs(val_Y_std - val_Y_std.mean())

print("MAE:", np.mean(std_error))


MAE: 0.03778143032920284


In [303]:
def simplemodel():
    model = Sequential()
    
    model.add(Dense(100, input_shape= (4,), kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(4)))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(4)))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, kernel_initializer='normal'))

    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    return model

In [304]:
snn = KerasRegressor(build_fn=simplemodel, epochs=100, verbose=True)
snn.fit(train_X_std, train_Y_std)

Epoch 1/100


C:\Users\Christopher\Anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


5080/5080 [==============================] - 2s 409us/step - loss: 0.1753
Epoch 2/100
5080/5080 [==============================] - 0s 81us/step - loss: 0.0726
Epoch 3/100
5080/5080 [==============================] - 0s 82us/step - loss: 0.0669
Epoch 4/100
5080/5080 [==============================] - 0s 88us/step - loss: 0.0631
Epoch 5/100
5080/5080 [==============================] - 0s 85us/step - loss: 0.0598
Epoch 6/100
5080/5080 [==============================] - 0s 82us/step - loss: 0.0552
Epoch 7/100
5080/5080 [==============================] - 0s 80us/step - loss: 0.0516
Epoch 8/100
5080/5080 [==============================] - 0s 88us/step - loss: 0.0486
Epoch 9/100
5080/5080 [==============================] - 0s 93us/step - loss: 0.0457
Epoch 10/100
5080/5080 [==============================] - 0s 80us/step - loss: 0.0440
Epoch 11/100
5080/5080 [==============================] - 0s 87us/step - loss: 0.0421
Epoch 12/100
5080/5080 [==============================] - 0s 92us/step - l

5080/5080 [==============================] - 0s 82us/step - loss: 0.0361
Epoch 97/100
5080/5080 [==============================] - 0s 77us/step - loss: 0.0361
Epoch 98/100
5080/5080 [==============================] - 0s 80us/step - loss: 0.0361
Epoch 99/100
5080/5080 [==============================] - 0s 78us/step - loss: 0.0361
Epoch 100/100
5080/5080 [==============================] - 0s 80us/step - loss: 0.0361


In [313]:
preds = snn.predict(val_X_std)

errors = abs(val_Y_std - preds)
print("MAE: ", round(np.mean(errors), 5))

# Rescale from the 0 - 1 scale
errors = abs(range_std_reverse(val_Y_std, df_thin.target.min(), df_thin.target.max()) - range_std_reverse(preds, df_thin.target.min(), df_thin.target.max()))
print("MAE Lambart Scale: ", round(np.mean(errors), 5))
# 0.1241036151090745 baseline

# Covnert to native scale
actual_lam = range_std_reverse(val_Y_std, df_thin.target.min(), df_thin.target.max())
pred_lam = range_std_reverse(val_Y_std.mean(),  df_thin.target.min(), df_thin.target.max())
predictions_lam = range_std_reverse(preds,  df_thin.target.min(), df_thin.target.max())


std_error = abs(actual_lam*np.exp(actual_lam) - pred_lam*np.exp(pred_lam) )
std_error_model = abs(actual_lam*np.exp(actual_lam) - predictions_lam*np.exp(pred_lam) )


print("MAE Native Scale Baseline:", np.mean(std_error))

print("Model MAE: ", np.mean(std_error_model) )

1270/1270 [==============================] - 0s 39us/step
MAE:  0.03743
MAE Lambart Scale:  0.03743
MAE Native Scale Baseline: 0.1115639153233884
Model MAE:  0.1108048787456431


730      (0.563716097900908+0j)
2131     (0.607760836636326+0j)
3070    (0.6184198843230663+0j)
4283    (0.6340379506406371+0j)
217      (0.506941831572853+0j)
3545     (0.623999725574594+0j)
4080     (0.631076236804234+0j)
3674    (0.6254499216217124+0j)
6176    (0.7149932866225828+0j)
4615    (0.6395628852916331+0j)
3459    (0.6228770190956557+0j)
4373     (0.635514570175142+0j)
465     (0.5420808814436198+0j)
5703    (0.6726735540671426+0j)
2981    (0.6176176731793163+0j)
1183    (0.5841679809726658+0j)
4040     (0.630334342774798+0j)
3680    (0.6254947610558818+0j)
1263    (0.5869092248849136+0j)
3622    (0.6247481567721125+0j)
3214    (0.6198757654709789+0j)
746     (0.5647203542073441+0j)
1611    (0.5969678823987247+0j)
4684    (0.6410297934334482+0j)
80      (0.4613716886460632+0j)
3784    (0.6266139965278106+0j)
2882    (0.6166521228858421+0j)
1616    (0.5970141616144082+0j)
6296    (0.7578394630322706+0j)
3612    (0.6247475962602763+0j)
                 ...           
509     

In [280]:
from scipy.special import lambertw

In [283]:
val_y_test = range_std_reverse(val_Y_std, norm_data.target_norm.min(), norm_data.target_norm.max())

val_y_test*np.exp(val_y_test)

730     3.117035e+09
2131    7.507361e+09
3070    1.002766e+10
4283    1.542625e+10
217     1.734977e+09
3545    1.171921e+10
4080    1.424669e+10
3674    1.220266e+10
6176    6.048353e+10
4615    1.780192e+10
3459    1.135736e+10
4373    1.603958e+10
465     2.379429e+09
5703    3.523446e+10
2981    9.806298e+09
1183    4.384284e+09
4040    1.396215e+10
3680    1.221789e+10
1263    4.627460e+09
3622    1.196646e+10
3214    1.044330e+10
746     3.162620e+09
1611    5.750513e+09
4684    1.846803e+10
80      1.309340e+09
3784    1.260375e+10
2882    9.547015e+09
1616    5.756674e+09
6296    8.645058e+10
3612    1.196627e+10
            ...     
509     2.498210e+09
5772    3.727436e+10
2153    7.507361e+09
4851    2.036251e+10
748     3.162647e+09
1617    5.756674e+09
5271    2.570949e+10
2404    8.221740e+09
5986    4.715180e+10
3313    1.077738e+10
2024    7.144944e+09
4546    1.731076e+10
5556    3.139891e+10
2181    7.625310e+09
471     2.386058e+09
5139    2.377025e+10
3961    1.354